In [ ]:
import numpy as np
import sys
import os
import plotly.graph_objects as go
import plotly.express as px
from toolkit.common.constants import *
from toolkit.cars.car_configuration import Car
from toolkit.lap.gps import *
from toolkit.mmd import MMD
from toolkit.steady_state_solver.ls_optimize import LS_Solver
from toolkit.steady_state_solver.iterative import Iterative_Solver
from toolkit.steady_state_solver.parachute import Parachute
from toolkit.common.maths import sa_lut

def generate_and_plot(params):
    print("Starting...")
    car = Car(front_ackermann = "nonlinear", camber_type = "combined", aero_type="simple")
    
    car.front_track = params[0] 
    car.rear_track = params[1]
    
    car.set_lltd()
    car.update_car()
    solver = Iterative_Solver()
    mmd = MMD(car, solver=solver, name=f"TW_F={twf:.2f} TW_R={twr:.2f}")
    mmd.mmd_sweep(v_avg, lin_space=use_lin, max_beta=max_beta, max_delta=max_delta, size=size, mu=0.65, long_g=val)
    mmd.plot_mmd(pub=True, lat=3, use_name=True, save_html=True)
    return mmd.name

In [ ]:
v_avg = 10
max_beta = 30
max_delta = 30
size = 30
use_lin = True

fz = 100
kappa = 0
i_a = np.deg2rad(0)
mu_corr = 0.65

# omega = np.deg2rad(0)
beta = np.deg2rad(0)

#delta_FL = np.deg2rad(10)
delta_FL = 0
delta_FR = delta_FL
delta_RL = 0
delta_RR = 0

In [ ]:
# Test slip angle
# sas = np.deg2rad(np.linspace(-10, 10, 5))
# tires = np.linspace(np.deg2rad(-6), np.deg2rad(6), 9)
# deltas = np.linspace(np.deg2rad(-6), np.deg2rad(6), 5)
omegas = np.linspace(np.deg2rad(0), np.deg2rad(11), 5)

# Make a default car
car = Car()

d = np.deg2rad(1.0)

for omega in omegas:
    print(np.rad2deg(omega))
    # calc slip angles
    delta = [d, d, 0, 0]
    tires = car.calculate_slip_angles(v_avg, omega, beta, delta[0], delta[1], delta[2], delta[3])
    # [safl, safr, sarl, sarr]
    # start with 0 for steer angle
    for i, tire_sa in enumerate(tires):
        fy, fx, m = car.steady_state_mmd(fz, tire_sa, kappa, v_avg, i_a, delta[i], mu = mu_corr, flip_s_a = False)
        # print(f"Slip Angle: {np.rad2deg(sa):.2f}")
        print(f"Slip Angle: {tire_sa}")
        print("Fy: {:.2f}, Fx: {:.2f}, M: {:.2f}".format(fy, fx, m))